In [ ]:
#Part_1_Text representation 

In [ ]:
import csv
import nltk
# Load the file
assignment_file = open('Assignment 1.csv',encoding="utf-8")
assignment_reader = csv.reader(assignment_file)
review_data = list(assignment_reader)

## 1. Tokenize each review in the collection
tokenized_review_list =[]
for review in review_data:
    review_token = nltk.word_tokenize(review[1]) #Only fetch 2nd column
    tokenized_review_list.append(review_token)
print(tokenized_review_list)

In [ ]:
#Convert tokens into lowercase
lower_tokenized_review_list =[]
for sub_list in tokenized_review_list:
    lower_tokenized_review_list.append(list(map(str.lower,sub_list)))

#lemmatization
lemmatizer = nltk.stem.WordNetLemmatizer()
lemmatized_lower_tokenized_review_list =[]
for sub_list in lower_tokenized_review_list:
    lemmatized_lower_tokenized_review_list.append([lemmatizer.lemmatize(token) for token in sub_list])
print(lemmatized_lower_tokenized_review_list)

In [ ]:
# Remove stop-words
from nltk.corpus import stopwords
stop_words_removed = []
for sub_list in lemmatized_lower_tokenized_review_list:
    result = [token for token in sub_list if not token in stopwords.words('english') if token.isalpha()] # remove punctuations
    stop_words_removed.append(result)
print(stop_words_removed)

In [ ]:
# Convert to TF-IDF vectors
from sklearn.feature_extraction.text import TfidfVectorizer
tdidf_review_list =[]
for sub_list in stop_words_removed:
    tdidf_review_list.append(" ".join(sub_list))
vectorizer = TfidfVectorizer(ngram_range=(1, 2), min_df=3)
vectorizer.fit(tdidf_review_list)
tfidfvectorized_review = vectorizer.transform(tdidf_review_list)
print(tfidfvectorized_review.toarray())

#Export to CSV file
tdidf_review= tfidfvectorized_review.toarray()
with open('assignment_1_solution_1_4.py.csv', 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    writer.writerows(tdidf_review)

In [ ]:
#POS TAG
POS_token_review_list =[]
for sub_list in tokenized_review_list:
    POS_token_review = nltk.pos_tag(sub_list)
    POS_token_temp = []
    for i in POS_token_review:
        POS_token_temp.append(i[0] + i[1])
    POS_token_review_list.append(" ".join(POS_token_temp))


# ID-IDF vectorization
vectorizer2 = TfidfVectorizer(min_df=4)
vectorizer2.fit(POS_token_review_list)
POS_vectorizer2 = vectorizer2.transform(POS_token_review_list)
print(POS_vectorizer2.toarray())

#Export to CSV file
pos_review = POS_vectorizer2.toarray()
with open('assignment_1_solution_1_5.py.csv', 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    writer.writerows(pos_review)

In [ ]:
# Part 2. Word embedding

In [ ]:
#choose the first 10 tokenized documents
tokenized_review_list_10 = tokenized_review_list[:10]
# do index-based encoding
words = [word for sub_list in tokenized_review_list_10 for word in sub_list]
from numpy import array
from sklearn.preprocessing import LabelEncoder
index_encoder = LabelEncoder()
encoded_index = index_encoder.fit(words)
encoded_index_vector = [index_encoder.transform(sub_list).tolist() for sub_list in tokenized_review_list_10]

#Export to CSV file

with open('assignment_1_solution_2_1.py.csv', 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    writer.writerows(encoded_index_vector)

In [ ]:
from sklearn.preprocessing import OneHotEncoder
index_list = [word for sub_list in encoded_index_vector for word in sub_list]
# make each word a list
index_list_vertical_array = array(index_list).reshape(-1,1)
# Do one-hot encoding
onehot_encoder = OneHotEncoder(sparse=False)
onehot_encoder = onehot_encoder.fit(index_list_vertical_array)
onehot_encoded_list = [onehot_encoder.transform(array(doc_i).reshape(-1, 1)).tolist() for doc_i in encoded_index_vector]
print(onehot_encoded_list)
# Export to CSV file which has column A as review index, column B as word index of a review, column C for vector index of a word and column D for value
with open('assignment_1_solution_2_2.csv', 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    for review_index in range(0, len(onehot_encoded_list)):
        for word_index in range(0, len(onehot_encoded_list[review_index])):
            for word_vector_index in range(0, len(onehot_encoded_list[review_index][word_index])):
                coordinate_and_value_list = []
                coordinate_and_value_list.append(review_index)
                coordinate_and_value_list.append(word_index)
                coordinate_and_value_list.append(word_vector_index)
                coordinate_and_value_list.append(onehot_encoded_list[review_index][word_index][word_vector_index])
                writer.writerow(coordinate_and_value_list)

In [ ]:
from gensim.scripts.glove2word2vec import glove2word2vec
# Loading the glove file
glove2word2vec(glove_input_file="glove.6B.50d.txt",word2vec_output_file="glove.6B.50d.txt.word2vec")
from gensim.models import KeyedVectors
glove_model = KeyedVectors.load_word2vec_format("glove.6B.50d.txt.word2vec",binary=False)
# Remove undefined word and map the review with glove file
glove_all_vector = []
undefined_word = []
for sub_list in tokenized_review_list_10:
    glove_all_word_vector =[]
    for word in sub_list:
        try:
            word_vector = glove_model[word]
            glove_all_word_vector.append(word_vector)
        except Exception:
            undefined_word.append(word)
    glove_all_vector.append(glove_all_word_vector)
print(glove_all_vector)

# Export to CSV file which has column A as review index, column B as word index of a review, column C for vector index of a word and column D for value
with open('assignment_1_solution_2_3.csv', 'w', newline='') as file:
    writer = csv.writer(file, quoting=csv.QUOTE_ALL)
    for review_index in range(0, len(glove_all_vector)):
        for word_index in range(0, len(glove_all_vector[review_index])):
            for word_vector_index in range(0, len(glove_all_vector[review_index][word_index])):
                coordinate_and_value_list = []
                coordinate_and_value_list.append(review_index)
                coordinate_and_value_list.append(word_index)
                coordinate_and_value_list.append(word_vector_index)
                coordinate_and_value_list.append(glove_all_vector[review_index][word_index][word_vector_index])
                writer.writerow(coordinate_and_value_list)